### Initialize variables

In [1]:
import os
from dotenv import load_dotenv
import pyodbc
import sqlalchemy as sa
from sqlalchemy import create_engine

from urllib.parse import quote_plus

load_dotenv()


server = os.getenv('SQL_SERVER_NAME')
database_name = os.getenv('SQL_DATABASE_NAME')
username = os.getenv('SQL_USER')
password = os.getenv('SQL_PASSWORD')

driver = '{ODBC Driver 17 for SQL Server}'

# Initiate Database connection (SQL Server)

In [2]:
conn = pyodbc.connect('DRIVER='+driver+';PORT=1433;SERVER='+server+';PORT=1443;DATABASE='+database_name+';UID='+username+';PWD='+ password)
cursor = conn.cursor()

# Initiate Semantic Kernel and Azure OpenAI Services

In [3]:
import semantic_kernel as sk
from semantic_kernel.connectors.ai.open_ai import (
    AzureChatCompletion,
    OpenAIChatCompletion,
)

kernel = sk.Kernel()

deployment, api_key, endpoint = sk.azure_openai_settings_from_dot_env()

kernel.add_chat_service(
    "chat_completion",
    AzureChatCompletion(deployment_name=deployment, endpoint=endpoint, api_key=api_key),
)



Kernel(plugins=KernelPluginCollection(plugins={}), prompt_template_engine=PromptTemplateEngine(), memory=NullMemory(), text_completion_services={'chat_completion': <function Kernel.add_text_completion_service.<locals>.<lambda> at 0x00000255AF9E2200>}, chat_services={'chat_completion': <function Kernel.add_chat_service.<locals>.<lambda> at 0x00000255AF9E2160>}, text_embedding_generation_services={}, default_text_completion_service='chat_completion', default_chat_service='chat_completion', default_text_embedding_generation_service=None, retry_mechanism=PassThroughWithoutRetry(), function_invoking_handlers={}, function_invoked_handlers={})

## Get Data

### Get schema and save it to a SQL table

In [4]:
# query to get the schema of the database
query_get_raw_schema = """
SELECT
    col.TABLE_SCHEMA + '.' + col.TABLE_NAME AS TABLE_NAME,
    col.COLUMN_NAME,
    col.DATA_TYPE,
    ISNULL(tc.CONSTRAINT_TYPE, 'NONE') AS CONSTRAINT_TYPE,
    col.IS_NULLABLE
FROM
    INFORMATION_SCHEMA.COLUMNS col
LEFT JOIN INFORMATION_SCHEMA.KEY_COLUMN_USAGE kcu
    ON col.COLUMN_NAME = kcu.COLUMN_NAME AND col.TABLE_NAME = kcu.TABLE_NAME AND col.TABLE_SCHEMA = kcu.TABLE_SCHEMA
LEFT JOIN INFORMATION_SCHEMA.TABLE_CONSTRAINTS tc
    ON kcu.CONSTRAINT_NAME = tc.CONSTRAINT_NAME AND kcu.TABLE_NAME = tc.TABLE_NAME AND kcu.TABLE_SCHEMA = tc.TABLE_SCHEMA
ORDER BY
    col.TABLE_SCHEMA,
    col.TABLE_NAME,
    col.ORDINAL_POSITION;

"""

cursor.execute(query_get_raw_schema)

columns = cursor.fetchall()

# iterate over the columns and insert them into the new table Adventure_Works_Schema or update them if they already exist
for column in columns:

    cursor.execute("""
MERGE dbo.Adventure_Works_Schema AS target
USING (SELECT ? AS TableName, ? AS ColumnName, ? AS DataType, ? AS ConstraintType, ? AS Nullable) AS source
ON (target.TableName = source.TableName AND target.ColumnName = source.ColumnName)
WHEN MATCHED THEN 
    UPDATE SET DataType = source.DataType, 
               ConstraintType = source.ConstraintType, 
               Nullable = source.Nullable
WHEN NOT MATCHED BY TARGET THEN 
    INSERT (TableName, ColumnName, DataType, ConstraintType, Nullable) 
    VALUES (source.TableName, source.ColumnName, source.DataType, source.ConstraintType, source.Nullable);
""", column.TABLE_NAME, column.COLUMN_NAME, column.DATA_TYPE, column.CONSTRAINT_TYPE, column.IS_NULLABLE)
    
# commit the changes to the DB
conn.commit()

### AOAI model analyze schema and user request

Get from Schema the following information: TableName, ColumnName and ConstraintType

In [5]:
# query to get the schema of the database
query_get_full_schema = """
SELECT TableName, ColumnName, ConstraintType
FROM dbo.Adventure_Works_Schema;
;
"""

cursor.execute(query_get_full_schema)

schema_columns = cursor.fetchall()

full_schema = ""
table_name = ""

# iterate over the columns and insert them into the new table Database_Schema
for column in schema_columns:

    if column.TableName == "Adventure_Works_Schema":
        continue
    
    if column.TableName != table_name:
        full_schema += " Table: " + f"{column.TableName} -"
        table_name = column.TableName
    full_schema += f" {column.ColumnName}{'(' + column.ConstraintType + ')' if column.ConstraintType != 'NONE' else ''}"

print(full_schema)



 Table: dbo.Adventure_Works_Schema - TableName ColumnName DataType ConstraintType Nullable Table: dbo.AdventureWorks_Schema - TableName ColumnName DataType ConstraintType Nullable Table: dbo.AWBuildVersion - SystemInformationID(PRIMARY KEY) Database Version VersionDate ModifiedDate Table: dbo.DatabaseLog - DatabaseLogID(PRIMARY KEY) PostTime DatabaseUser Event Schema Object TSQL XmlEvent Table: dbo.ErrorLog - ErrorLogID(PRIMARY KEY) ErrorTime UserName ErrorNumber ErrorSeverity ErrorState ErrorProcedure ErrorLine ErrorMessage Table: HumanResources.Department - DepartmentID(PRIMARY KEY) Name GroupName ModifiedDate Table: HumanResources.Employee - BusinessEntityID(PRIMARY KEY) NationalIDNumber LoginID OrganizationNode OrganizationLevel JobTitle BirthDate MaritalStatus Gender HireDate SalariedFlag VacationHours SickLeaveHours CurrentFlag rowguid ModifiedDate Table: HumanResources.EmployeeDepartmentHistory - BusinessEntityID(PRIMARY KEY) DepartmentID(PRIMARY KEY) ShiftID(PRIMARY KEY) StartD

Model to look over the schema and user query and decide what tables and columns are relevant.

In [6]:
sk_prompt_analyze_schema = """
Analyze the schema of the database and decide which tables and columns are relevant for answering the user's request. Schema: {{$schema}}, User request {{$user_request}}.
Output solely the SQL query for SQL server.
"""

chat_function = kernel.create_semantic_function(
    prompt_template=sk_prompt_analyze_schema,
    function_name="ChatBot",
    max_tokens=2000,
    temperature=0.4,
    top_p=0.5,
)

context = kernel.create_new_context()
context["schema"] = full_schema
context["user_request"] = "I want to see what are the names of top 5 products best selling in our catalog"

bot_answer = await chat_function.invoke(context=context)

print(bot_answer)

SELECT TOP 5 p.Name
FROM Production.Product p
JOIN Sales.SalesOrderDetail sod ON p.ProductID = sod.ProductID
GROUP BY p.Name
ORDER BY SUM(sod.OrderQty) DESC;


## Retrieve data required to answer user request

In [7]:
# query to get the schema of the database
query_get_answer_data = bot_answer.result

cursor.execute(query_get_answer_data)

answer_raw_data = cursor.fetchall()

answer_data = ""

# iterate over the columns and insert them into the new table Database_Schema
for data in answer_raw_data:    
    answer_data += str(data) + "\n"

print(answer_data)


('AWC Logo Cap',)
('Water Bottle - 30 oz.',)
('Sport-100 Helmet, Blue',)
('Long-Sleeve Logo Jersey, L',)
('Sport-100 Helmet, Black',)



## Generate answer back to the user based on retrieved data

In [8]:
sk_prompt_reply_user = """
Use the given information (retrieved_data) to answer the user's request. User request: {{$user_request}}. Retrieved_data: {{$retrieved_data}}.
"""

reply_function = kernel.create_semantic_function(
    prompt_template=sk_prompt_reply_user,
    function_name="ChatBot",
    max_tokens=2000,
    temperature=0.7,
    top_p=0.5,
)

context["retrieved_data"] = answer_data

bot_answer = await reply_function.invoke(context=context)

print(bot_answer)

The top 5 best selling products in our catalog are: 
1. AWC Logo Cap 
2. Water Bottle - 30 oz. 
3. Sport-100 Helmet, Blue 
4. Long-Sleeve Logo Jersey, L 
5. Sport-100 Helmet, Black.


---------------------------------------------------- DRAFT ------------------------------------------------------------------------------
## Generate Queries

In [33]:
sk_prompt = """
Generate a SQL query for a SQL Server database to retrieve data from an ERP system. Database name is called: {{$database}}
Take into consideration the schema for all tables: {{$schema}}
The query should return the required data to satisfy the request of the user: {{$user_request}}"""

In [35]:
chat_function = kernel.create_semantic_function(
    prompt_template=sk_prompt,
    function_name="ChatBot",
    max_tokens=2000,
    temperature=0.4,
    top_p=0.5,
)

context = kernel.create_new_context()
context["database"] = database_name
context["schema"] = schema
context["user_request"] = "I want to see what are the top 5 products in our catalog"

bot_answer = await chat_function.invoke(context=context)

print(bot_answer)


Error: (<ErrorCodes.ServiceError: 6>, "<class 'semantic_kernel.connectors.ai.open_ai.services.azure_chat_completion.AzureChatCompletion'> service failed to complete the prompt", BadRequestError('Error code: 400 - {\'error\': {\'message\': "This model\'s maximum context length is 8192 tokens. However, your messages resulted in 9459 tokens. Please reduce the length of the messages.", \'type\': \'invalid_request_error\', \'param\': \'messages\', \'code\': \'context_length_exceeded\'}}'))
